In [3]:
import pymc as pm
import numpy as np
import arviz as az

%load_ext lab_black
%load_ext watermark

# Lister

This example demonstrates ...

Adapted from [unit 10: lister.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit10/lister.odc).

Data can be found [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/r.txt).

## Associated lecture video: Unit 10 Lesson 1

In [1]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed?v=xomK4tcePmc&list=PLv0FeK5oXK4l-RdT6DWJj0_upJOG2WKNO&index=98" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>

## Problem statement

Sir Joseph Lister (1827--1912), Professor of Surgery at Glasgow University, influenced by Pasteur's ideas, found that a wound wrapped in bandages treated by Carbolic acid (phenol) would often not become infected.

Here are Lister's data on treating open fractures and amputations:

Period    Carbolic acid used    Results

1864--1866     No       Treated   34 patients, 15 died and 19 recovered

1867--1870     Yes     Treated   40 patients,   6 died and 34 recovered


Estimate and interpret the risk difference,  risk ratio, and   odds ratio


notes:
- again with the Wishart! explain here instead, since this one is actually first
- will need to talk about LKJ distribution and equivalence
- possibly useful paper:
    - https://arxiv.org/pdf/1809.04746.pdf


In [11]:
n1 = 34
y1 = 15
n2 = 40
y2 = 6
mu0 = np.array([-0.5, -1.5])
S = np.array([[0.1, 0], [0, 0.1]])

In [39]:
with pm.Model() as m:
    chol, corr, stds = pm.LKJCholeskyCov(
        "chol", n=2, eta=1, sd_dist=pm.HalfCauchy.dist(1, shape=2), compute_corr=True
    )

    mu = pm.MvNormal("mu", mu0, chol=chol)

    p1 = mu[0]
    p2 = mu[1]

    pm.Binomial("y1", n=n1, logit_p=p1, observed=y1)
    pm.Binomial("y2", n=n2, logit_p=p2, observed=y2)

    pm.Deterministic("rd", p1 - p2)
    pm.Deterministic("rr", p1 / p2)
    pm.Deterministic("or", (p1 / (1 - p1)) / (p1 / (1 - p2)))
    trace = pm.sample(3000)

Auto-assigning NUTS sampler...
INFO:pymc:Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
INFO:pymc:Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
INFO:pymc:Multiprocess sampling (4 chains in 4 jobs)
NUTS: [chol, mu]
INFO:pymc:NUTS: [chol, mu]


/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/aesara/compile/function/types.py:962: RuntimeWarning: invalid value encountered in accumulate
  self.fn()
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/aesara/compile/function/types.py:962: RuntimeWarning: invalid value encountered in accumulate
  self.fn()
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/aesara/compile/function/types.py:962: RuntimeWarning: invalid value encountered in accumulate
  self.fn()
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/aesara/compile/function/types.py:962: RuntimeWarning: invalid value encountered in accumulate
  self.fn()
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarn

In [40]:
az.summary(trace)

/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/arviz/stats/diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
mu[0],-0.352,0.270,-0.804,0.213,0.008,0.006,1166.0,1200.0,1.00
mu[1],-1.659,0.329,-2.383,-1.112,0.008,0.006,1670.0,3176.0,1.00
chol[0],0.813,1.656,0.026,2.232,0.032,0.023,838.0,596.0,1.01
chol[1],-0.107,1.705,-1.336,1.375,0.078,0.056,2087.0,1206.0,1.01
chol[2],0.748,1.781,0.036,1.925,0.074,0.052,250.0,68.0,1.02
"chol_corr[0, 0]",1.000,0.000,1.000,1.000,0.000,0.000,12000.0,12000.0,NaN
"chol_corr[0, 1]",-0.036,0.573,-0.980,0.855,0.014,0.012,1076.0,315.0,1.01
"chol_corr[1, 0]",-0.036,0.573,-0.980,0.855,0.014,0.012,1076.0,315.0,1.01
"chol_corr[1, 1]",1.000,0.000,1.000,1.000,0.000,0.000,3272.0,3740.0,1.00
chol_stds[0],0.813,1.656,0.026,2.232,0.032,0.023,838.0,596.0,1.01
